# Prudential Life Insurance Assessment

In [1]:
import math
import numpy as np
import pandas as pd
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
plt.style.use('seaborn-whitegrid')

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Get the Data

In [2]:
df = pd.read_csv('E:/AdvAnal/python_103/1_Data/14_dtree_train.csv')
dft = pd.read_csv('E:/AdvAnal/python_103/1_Data/14_dtree_test.csv')

In [3]:
test = df[df.index < 30000].copy()
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 0 to 29999
Data columns (total 7 columns):
y        30000 non-null int64
21_df    30000 non-null int64
22_df    30000 non-null int64
23_df    30000 non-null int64
24_df    30000 non-null int64
25_df    30000 non-null int64
26_df    30000 non-null int64
dtypes: int64(7)
memory usage: 1.8 MB


In [4]:
train = df[df.index >= 30000].copy()
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29381 entries, 30000 to 59380
Data columns (total 7 columns):
y        29381 non-null int64
21_df    29381 non-null int64
22_df    29381 non-null int64
23_df    29381 non-null int64
24_df    29381 non-null int64
25_df    29381 non-null int64
26_df    29381 non-null int64
dtypes: int64(7)
memory usage: 1.8 MB


In [5]:
sub = dft
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19765 entries, 0 to 19764
Data columns (total 7 columns):
Id       19765 non-null int64
21_df    19765 non-null int64
22_df    19765 non-null int64
23_df    19765 non-null int64
24_df    19765 non-null int64
25_df    19765 non-null int64
26_df    19765 non-null int64
dtypes: int64(7)
memory usage: 1.1 MB


# Clean the Data

In [6]:
cat_feats = ['21_df', '22_df', '23_df', '24_df', '25_df', '26_df']

In [7]:
y_train = train['y'].copy()
x_train = train.drop('y', 1).copy()
x_train = pd.get_dummies(x_train,columns=cat_feats,drop_first=True).copy()

In [8]:
y_test = test['y'].copy()
x_test = test.drop('y', 1).copy()
x_test = pd.get_dummies(x_test,columns=cat_feats,drop_first=True).copy()

In [9]:
y_df = df['y'].copy()
x_df = df.drop('y', 1).copy()
x_df = pd.get_dummies(x_df,columns=cat_feats,drop_first=True).copy()

In [10]:
y_sub = sub['Id'].copy()
x_sub = sub.drop('Id', 1).copy()
x_sub = pd.get_dummies(x_sub,columns=cat_feats,drop_first=True).copy()

# KNN

In [11]:
error_rate = []
for i in range(6,61,5):    
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(x_train,y_train)
    pred_i = knn.predict(x_test)
    error_rate.append(np.mean(pred_i != y_test))

Text(0,0.5,'Error Rate')

In [12]:
plt.figure(figsize=(8,4))
plt.plot(range(6,61,5),error_rate,color='darkred', marker='o',markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')

In [13]:
knn = KNeighborsClassifier(n_neighbors=20)
knn.fit(x_train,y_train)
y_pred=knn.predict(x_test)

In [14]:
test_calc = pd.concat([pd.DataFrame(y_test).reset_index(drop=True),pd.DataFrame(y_pred).reset_index(drop=True)],axis=1)
test_calc.rename(columns={0: 'predicted'}, inplace=True)

df_table = confusion_matrix(test_calc['y'],test_calc['predicted'])
print (df_table)

print('accuracy:', (df_table[0,0] + df_table[1,1]) / (df_table[0,0] + df_table[0,1] + df_table[1,0] + df_table[1,1]))
print ('precision:', df_table[1,1] / (df_table[1,1] + df_table[0,1]))
print('recall:', df_table[1,1] / (df_table[1,1] + df_table[1,0]))

p = df_table[1,1] / (df_table[1,1] + df_table[0,1])
r = df_table[1,1] / (df_table[1,1] + df_table[1,0])
print('f1 score: ', (2*p*r)/(p+r))

[[ 688  355   19  159  232  606  348  750]
 [ 340  848   16   97  333  611  321  711]
 [  44   39   84  194   44   42   30   23]
 [  20    3   18  455    3   56   55  107]
 [ 166  251    4    5 1179  542  227  378]
 [ 264  224    1   21  288 3059  622 1127]
 [ 158   60    1    9   36  689 1692 1420]
 [  43   36    0   39   29  449  383 8947]]
accuracy: 0.688480502017
precision: 0.704904405653
recall: 0.713804713805
f1 score:  0.709326641573
